In [1]:
from mido import MidiFile
import warnings
warnings.filterwarnings("ignore")
from commons import *
import pandas as pd
import numpy as np
import os, sys
from contextlib import suppress
import pickle  
from sklearn.preprocessing import StandardScaler

In [2]:
files_path = str(sys.path[0]) + '\Full_MIDI'

dict = {}
count_files = 0
      
for root, dirs, files in os.walk(files_path, topdown=False):
    for name_file in files:
        dict[count_files] = {"file": os.path.join(root, name_file), "file_name": name_file}
        count_files = count_files + 1

df_files = pd.DataFrame.from_dict(dict, "index")
df_files.sort_values('file_name',inplace=True)
df_files.reset_index(inplace=True,drop=True)
display(df_files)

,file,file_name
0,C:\dh\FULL_MIDI\Full_MIDI\Categories\unclassif...,'74-'75.mid
1,C:\dh\FULL_MIDI\Full_MIDI\Categories\Classical...,'Lieder ohne Worte' Book 2 op30 n3.mid
2,C:\dh\FULL_MIDI\Full_MIDI\Categories\Pop_and_T...,(Dj Quicksilver) bellisima.mid
3,C:\dh\FULL_MIDI\Full_MIDI\Categories\unclassif...,(TEARS).MID
4,C:\dh\FULL_MIDI\Full_MIDI\Categories\2009 MIDI...,(Your Love Keeps Lifting Me) Higher and Higher...
...,...,...
88230,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zyysayr.mid,zyysayr.mid
88231,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zz_top-shes_just...,zz_top-shes_just_killing_me.mid
88232,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzdq.MID,zzdq.MID
88233,C:\dh\FULL_MIDI\Full_MIDI\Z\Z\zzgsv.mid,zzgsv.mid


In [3]:
BATCH_SIZE = 25
files_from = 0
files_to = 25
CSV_SAVE = 'df_scaled'
print(files_from,files_to)

0 25


In [4]:
dict = {}
count_files = 0
count_batch = 0
errors_arr = []

In [5]:
for index, row in df_files[files_from:files_to].iterrows():
    with suppress(Exception):
        if ((count_files % BATCH_SIZE == 0) and (count_files > 0)):
            
            SaveAllFiles(dict, files_path)
            
            del df_midi_stats
            count_batch = count_batch + 1

        dict[count_files] = get_theme_stats(row.file, row.file_name)[0] 

        count_files = count_files + 1
        print('success', str(count_files), '/', index + 1)
        print('batch', str(count_batch))

SaveAllFiles(dict, files_path)

.Study No.3, Retour du pâtre, opus.109.mid G
sec: 91.04562980078107
comp: 37
success 25 / 25
batch 0
SAVING!!
SAVED!!
